In [1]:
from utils.read_file import *

ERROR! Session/line number was not unique in database. History logging moved to new session 108


In [2]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

INSURANCE_PATH = "競賽資料集/reference/insurance"

FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"
FINANCE_PATH = "競賽資料集/reference/finance"

In [3]:
question = read_json(QUESTION_PATH)["questions"]

In [4]:
# from huggingface_hub import snapshot_download

# # 下載模型到指定路徑
# local_model_path = "./models/bge-m3"
# snapshot_download(repo_id="BAAI/bge-m3", revision="main", local_dir=local_model_path)

In [ ]:
from sentence_transformers import SentenceTransformer

# 加載模型
embbeded_model = SentenceTransformer("./models/bge-m3", device='cuda')

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_ans(embedded_model, question_data: dict, documents: list, k=1):
    document_ebeddings = embedded_model.encode(documents)

    # 使用者查詢
    user_query = question_data["query"]

    # 查詢文本轉換成嵌入向量
    query_embedding = embbeded_model.encode([user_query])

    # 計算相似度
    similarities = cosine_similarity(query_embedding, document_ebeddings)
    k_highest = np.argsort(similarities[0])[-k:][::-1]
    return k_highest, np.sort(similarities[0])[::-1][:k]

### 驗證faq資料集的正確率

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 每段的目標長度
    chunk_overlap=400  # 分段間的重疊字符數
)


def validate_finance(embbeded_model, question):
    ans = read_json(ANS_PATH)
    correct = 0
    half_correct = 0 #答案在前五

    for i in range(50, 100):
        src = question[i]["source"]
        document, order_list = read_target_insurance_pdf(FINANCE_PATH, src)
        tokens = []
        chunk_real_index = []
        for j in range(len(document)):
            text = document[j]
            chunks = text_splitter.split_text(text)
            tokens.extend(chunks)
            chunk_real_index.extend([j + k * 0 for k in range(len(chunks))])

                
        chunk_real_index=np.array(chunk_real_index)


        token_index, similarities = get_ans(embbeded_model, question[i], tokens, 5)
        real_index = chunk_real_index[token_index]
        all_predict = order_list[real_index]
        predict = all_predict[0]


        if (predict == ans["ground_truths"][i]["retrieve"]):
            correct += 1
        else:
            print(f"qid: {i + 1}, predict: {predict}, ans: {ans["ground_truths"][i]["retrieve"]}")
            chunks_text = f"qid: {i + 1}, ans:{ans["ground_truths"][i]["retrieve"]}\n\n\n"

            for j in range(len(token_index)):
                chunks_text += f"index: {order_list[real_index[j]]}, similarity: {similarities[j]}\n\n{tokens[token_index[j]]}\n\n\n"

            chunks_text += f"正確答案{ans["ground_truths"][i]["retrieve"]}的token\n"
            for j in range(len(tokens)):
                
                if order_list[chunk_real_index[j]] == ans["ground_truths"][i]["retrieve"]:
                    chunks_text += f"token:{j + 1}\n\n{tokens[j]}\n\n\n"


            with open(f"output_finance/qid_{i + 1}.txt", 'w', encoding='utf-8') as file:
                file.write(chunks_text)
        if (ans["ground_truths"][i]["retrieve"] in all_predict):
            half_correct += 1

    print(f"acc: {correct / 50 * 100} %")
    print(f"in rank 5: {half_correct / 50 * 100} %")

### 驗證finance資料集的正確率

In [8]:
import os

if not os.path.exists("output_finance/"):
    os.makedirs("output_finance/")
validate_finance(embbeded_model, question)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


qid: 53, predict: 145, ans: 351
qid: 58, predict: 112, ans: 209
qid: 61, predict: 870, ans: 900
qid: 62, predict: 1024, ans: 591
qid: 64, predict: 706, ans: 124
qid: 67, predict: 837, ans: 1021
qid: 69, predict: 545, ans: 981
qid: 70, predict: 298, ans: 490
